## CNN untuk data gempabumi 
## Klasifikasi noise dan event gempabumi


In [ ]:
#init work folder

import os #library untuk mengakses atau berinteraksi dengan Operating System laptop kita

global directory
directory = os.getcwd()

#print (directory)

In [ ]:
#baca semua dataset di folder dataset (noise dan gempa)
#kita buat fungsi untuk membuat labels dataset dengan output files csv "labels.csv"

import pandas as pd #library untuk penanganan data tabel

def make_labels_csv():
    
    #jika ada file labels.csv hapus lebih dahulu 
    if os.path.exists(directory+'/labels.csv'):
        os.remove(directory+'/labels.csv')

    #headar tabel berupa nama file dan labelsnya dan simpan dalam bentuk csv
    df = pd.DataFrame(columns=['files_name', 'labels'])
    df.to_csv(directory+'/labels.csv', mode='w', header=True, index=False)

    #lakukan looping pada setiap folder lalu simpan 
    for r,d,files in os.walk(directory+'/dataset/event/'):
        i = 0
        while i < len(files):
            data_ = {
                'files_name'    :   [files[i]],
                'labels'        :   ['event']
                }
            i += 1

            df = pd.DataFrame(data_)
            df.to_csv(directory+'/labels.csv', mode='a', header=False, index=False)

    for r,d,files in os.walk(directory+'/dataset/noise/'):
        i = 0
        while i < len(files):
            data_ = {
                'files_name'    :   [files[i]],
                'labels'        :   ['noise']
                }
            i += 1

            df = pd.DataFrame(data_)
            df.to_csv(directory+'/labels.csv', mode='a', header=False, index=False)

#make labels
make_labels_csv()

## Cek data waveform mseed menggunakan library obspy
## yang perlu dicek adalah data array jumlah samples nya, ini akan berpengaruh pada input model
## Pada inputan model CNN, 1D atau pun 2D harus memiliki bentuk (Shape) yang sama antar data.
## Sehingga untuk memudahkan inputan data pada model CNN, alternatif cara yang dilakukan ialah merubah bentuk data (Spectogram/Image atau Array )

In [ ]:
from obspy import read
import matplotlib.pyplot as plt

def readwaveform(path):
    if os.path.exists(directory+'/dataset/spectogram/'):
        os.rmdir(directory+'/dataset/spectogram/')
    
    os.mkdir(directory+'/dataset/spectogram/')

    #baca csv dan inisiasi header dan ubah kedalam bentuk array
    read_csv = pd.read_csv(directory+'/'+path)
    init_stations = read_csv[['files_name','labels']]
    arr_init_stations = init_stations.to_numpy()
    
    for i in arr_init_stations:
        st = read(directory+'/dataset/'+i[1]+'/'+i[0]).resample(100)
        # print(st.__str__(extended=True))
        data = st[0].data.astype('float32')
        sr = int(st[0].stats.sampling_rate)
        
        # print(data)
        # rubah bentuk domain waktu ke domain frekuensi menggunkan spectogram
        fig, ax = plt.subplots(figsize=(3,2))
        ax.specgram(data,Fs=100,NFFT=256,cmap='gray',vmin=-10,vmax=25);
        ax.set_xlim([0,60])
        ax.axis('off')
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
        plt.margins(0,0)
        plt.savefig(directory+'/dataset/spectogram/'+i[0]+'.png',bbox_inches='tight',transparent = True,pad_inches=0,dpi=50) #simpan kedalam folderxs
        plt.close()
            
        
readwaveform('labels.csv')


In [ ]:
import tensorflow as tf

class HParams(object):
    """ Hparams was removed from tf 2.0alpha so this is a placeholder
    """
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

@tf.function
def _normalize_tensorflow(S, hparams):
        return tf.clip_by_value((S - hparams.min_level_db) / -hparams.min_level_db, 0, 1)

def _tf_log10(x):
        numerator = tf.math.log(x)
        denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
        return numerator / denominator

def _amp_to_db_tensorflow(x):
        return 20 * _tf_log10(tf.clip_by_value(tf.abs(x), 1e-5, 1e100))

def _stft_tensorflow(signals, hparams):
        return tf.signal.stft(
            signals,
            hparams.win_length,
            hparams.hop_length,
            hparams.n_fft,
            pad_end=True,
            window_fn=tf.signal.hann_window,
        )

def spectrogram_tensorflow(y, hparams):
        D = _stft_tensorflow(y, hparams)
        S = _amp_to_db_tensorflow(tf.abs(D)) - hparams.ref_level_db
        return _normalize_tensorflow(S, hparams)


def spectogramtwo(path):
    if os.path.exists(directory+'/dataset/spectogram_2/'):
        os.rmdir(directory+'/dataset/spectogram_2/')
    
    os.mkdir(directory+'/dataset/spectogram_2/')
    
    read_csv = pd.read_csv(directory+'/'+path)
    init_stations = read_csv[['files_name','labels']]
    arr_init_stations = init_stations.to_numpy()
    
    for i in arr_init_stations:
        st = read(directory+'/dataset/'+i[1]+'/'+i[0]).resample(100)
            # print(st.__str__(extended=True))
        data = st[0].data.astype('float32')
        sr = int(st[0].stats.sampling_rate)

        hparams = HParams(  
                # spectrogramming
                win_length = 256,
                n_fft = 256,
                hop_length= 128,
                ref_level_db = 250000,
                min_level_db = -250000,
                # mel scaling
                num_mel_bins = 128,
                mel_lower_edge_hertz = 0,
                mel_upper_edge_hertz = (sr/2),
                # inversion
                power = 1, # for spectral inversion
                griffin_lim_iters = 50,
                pad=True,
                #
            )
        
        spectrogram = spectrogram_tensorflow(data.astype('float32'), hparams)
        fig, ax = plt.subplots(ncols=1, figsize=(15,4))
        cax = ax.matshow(spectrogram.numpy().T, aspect='auto', origin='lower')
        # fig.colorbar(cax)
        ax.axis('off')
        plt.savefig(directory+'/dataset/spectogram_2/'+i[0]+'.png', bbox_inches='tight', dpi=50)
        plt.close
        
        
spectogramtwo('labels.csv')

In [ ]:
train_df = pd.read_csv('labels.csv')
train_df= train_df[['files_name','labels']]
train_df['spectogram'] = train_df.files_name.map(lambda id: f'dataset/spectogram_2/{id}.png')
train_df['labels_num'] = train_df.labels.map(lambda x : 1 if x == 'event' else 0) 
# print(train_df.head())

print(train_df['spectogram'])
print(train_df['labels_num'])

In [ ]:
## Baca image untuk mendapatkan informasi bentuk
from skimage.io import imread
import cv2
import numpy as np

def preprocess_spectogram(path):
    images = []
    for file_path in path:
        img = imread(file_path)
        width = int(img.shape[1])
        height = int(img.shape[0])
        dim = (width, height)
        img_resize = cv2.resize(img, dim)
        images.append(img_resize)
    images = np.asarray(images, dtype=np.float32)
    images = images / 255.0
    images.shape
    # images = images.reshape(images.shape[1], images.shape[2],1)
    
    input_shape = images.shape 
    # print(f'image is {channels} \n {images.shape}')

    return images, input_shape
    
#ambil nilai shape spectogram
images, input_shape = preprocess_spectogram(train_df.spectogram.values)

# print (images)


In [ ]:
from tensorflow.keras import layers
#input shape
shapes_spec = (images.shape[1],images.shape[2],4)
input_shape_spec =layers.Input(shape= shapes_spec)

print(input_shape_spec)


In [ ]:
##menentukan data dan label yang akan digunakan training
from tensorflow.keras.utils import to_categorical

x_train_spec = images

# labels - convert class vectors to binary class matrices One Hot Encoding
labels = train_df.labels_num.values
labels = to_categorical(labels, 2)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train_spec, labels, test_size = 0.2, random_state=666)

In [ ]:
## Buat fungsi model CNN 
import tensorflow as tf
from tensorflow.keras import layers, models


def cnnmodels(input_shape_spec):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(shapes_spec)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))
    
    return(model)

In [ ]:
model = cnnmodels(input_shape_spec)
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
## training model CNN
opt = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics='accuracy')

In [ ]:

epochs = 50
batch_size = 64

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

opt = tf.keras.optimizers.Adam(learning_rate=1e-6)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1,
)

In [ ]:
# load weights
model.load_weights('best_model.h5')

final_loss, final_acc = model.evaluate(x_test, y_test, verbose=1)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))  

In [ ]:
final_loss, final_acc = model.evaluate([x_test], y_test,batch_size=128, verbose=1)
print("\nTest data, accuracy: {:5.2f}%".format(100*final_acc))
print("\nTest data, loss: {:5.2f}%".format(100*final_loss))

In [ ]:
# Confusiion Matrix
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

y_pred = model.predict([x_test]) 
y_pred = np.argmax(model.predict([x_test]),axis=1) 
predicted_test = np.argmax(y_test, axis=1)
cm = confusion_matrix(predicted_test, y_pred)

print(cm)

In [ ]:
print (classification_report(predicted_test, y_pred))

In [ ]:
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(7,7))
ax.plot(history.history['accuracy'])
ax.plot(history.history['val_accuracy'])
ax.set_title('Model Accuracy')
ax.set_ylabel('Accuracy')
ax.set_xlabel('Epoch')
ax.legend(['train','test'])
plt.show()

In [ ]:
plt.style.use('default')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Event','Noise'])
disp.plot(cmap='Blues', values_format='')
plt.title(f'Classification CNN Results ({epochs} epochs)')
plt.tight_layout()
plt.savefig('M1(confusion_matrix).png')
plt.show()

In [ ]:
predicted_df = pd.read_csv('/Users/litbanggeo/Documents/Kerjaan_litbang/Kegiatan_2023/BAHAN_AJAR_PUSDIKLAT26AGUSTUS23/eq_cnn/labels.csv')
predicted_df['spectogram'] = predicted_df.files_name.map(lambda id: f'dataset/spectogram_2/{id}.png')

imagesP, input_shapeP = preprocess_spectogram(predicted_df.spectogram.values)

classTarget = [
    'Event', 
    'Noise', 
]

y_pred = model.predict(imagesP) 
print(' prediksi data adalah ## {} ##'.format(classTarget[np.argmax(y_pred[103])]))
